In [26]:
#;.pykx.disableJupyter()

PyKX now running in 'python' mode (default). All cells by default will be run as python code. 
Include '%%q' at the beginning of each cell to run as q code. 


In [27]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


##### Learning outcomes

To understand: 
* Structure of a splayed database
* Saving splayed tables using .Q.en
* Loading splayed tables into memory
* Operating on splayed tables
* Guidance on the sym file
* Considerations on splayed tables

# Introduction
When a table is too large to fit into memory, we can break it into pieces and save each column individually under a directory with the table name - this is what we call splaying a table. This resolves two main issues that we will encounter if we save our table as a flat file:

* A flat file table must fit into memory. By splaying a table, the table is mapped into memory
* Operations on flat files might be slow due to reloading the entire table into memory each time. By splaying a table, columns are loaded on demand which increases the performance of queries. 

# Structure of a splayed database

Splay tables are particular in their on-disk structure and certain files (.e.g the `.d` file) will determine the way in which the table is structured in memory. 

Inside the directory each column is saved in a separate file under the same name as the column name. Each column is saved as a list of corresponding type in a kdb+ binary flat file. The order of the columns is saved in a separate file, called `.d`, in the same directory. `.d` is a kdb+ binary file containing a symbol list. Let's look at how the directory structure would look:

<img src="../images/splayedTable.png" width="500" height="500">

##### The .d file

The `.d` file present within a splayed table stores the information on the order in which the columns appear in the table. For a table `flatT`, we have the following meta: 

In [13]:
t:([]sym:`TSLA`IBM`MS`GM;size:20 30 40 50;price:1.1 2.2 3.3 4.4)
meta t
\ls

c    | t f a
-----| -----
sym  | s    
size | j    
price| f    
"Practical Guidance- Splayed Tables.ipynb"
"Splayed Tables Exercises.ipynb"
"Splayed Tables.ipynb"


If this were saved splayed on disk, the .d file would consist of the following symbol list: <code>\`sym\`size\`price</code>. If we modified this file to reorder the columns that would be the new table structure. 

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Removing columns from the .d files is a sort-of "safe" deletion to do before clearing off the actual data!</i></p>

##### Sym file 

There is a difference between the `sym` file located in the db directory and the `sym` files located in the trade and quote directory.

1. The sym file located in the db directory consists of all the distinct symbols in both the trade and quote table. For example, ``` `BAC`BTU`DIS`GE`T`NYSE`NASDAQ`LSE`JPX ```
2. The sym file located in the trade and quote directory consists of a numeric index list that corresponds to the sym file located in the db directory. These values allow kdb+ to map the correct symbols to each of the rows for the different tables.

# Creating a splayed table
A table can be saved as splayed using `set`, as for flat files, but for the table to be saved
as splayed the file path should end with a forward slash:

``` `:path_to_file/filename/ set tablename```

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If we want to save a table in a splayed format our filepath has to end with a forward slash <code>/</code></i></p>

The underlying kdb+ table must be un-keyed and any symbol columns must be enumerated in order to successfully save a splayed table. 

## Creating a splayed table without symbol columns

Let's consider a table that is un-keyed and has no symbol column. 

In [15]:
n:10000
10#tab:([]timestamp:asc n?.z.P;price:n?500f; size:n?100 200 500 1000) //creating a trade table

timestamp                     price    size
-------------------------------------------
2000.01.01D01:56:54.531680879 311.5894 500 
2000.01.01D13:34:24.561852264 198.4798 200 
2000.01.02D14:39:07.012254348 234.7829 1000
2000.01.03D01:04:40.990981917 170.0476 500 
2000.01.03D17:17:57.272462255 487.053  1000
2000.01.04D05:54:30.271955657 455.2298 100 
2000.01.06D04:51:06.319768444 416.9192 100 
2000.01.06D13:18:43.794450740 220.4627 500 
2000.01.07D08:25:31.248157151 383.3816 200 
2000.01.07D12:35:34.665080890 191.4548 200 


In [16]:
`:tab/ set tab   //we have created our table!

:tab/


In [17]:
key `:tab      //the files in our new trade folder
\ls tab

`s#`.d`price`size`timestamp
"price"
"size"
"timestamp"


In [18]:
get `:tab

timestamp                     price    size
-------------------------------------------
2000.01.01D01:56:54.531680879 311.5894 500 
2000.01.01D13:34:24.561852264 198.4798 200 
2000.01.02D14:39:07.012254348 234.7829 1000
2000.01.03D01:04:40.990981917 170.0476 500 
2000.01.03D17:17:57.272462255 487.053  1000
2000.01.04D05:54:30.271955657 455.2298 100 
2000.01.06D04:51:06.319768444 416.9192 100 
2000.01.06D13:18:43.794450740 220.4627 500 
2000.01.07D08:25:31.248157151 383.3816 200 
2000.01.07D12:35:34.665080890 191.4548 200 
2000.01.08D03:54:07.521201137 6.230592 100 
2000.01.13D17:37:56.226649925 261.6942 100 
2000.01.14D03:01:43.148033934 355.778  500 
2000.01.15D16:30:39.739352136 253.8772 1000
2000.01.16D21:05:45.867885226 232.6473 500 
2000.01.17D01:19:26.280580411 226.995  1000
2000.01.17D15:05:03.288834384 215.1588 1000
2000.01.17D18:41:00.692772457 250.7397 1000
2000.01.18D10:05:49.082921719 448.2771 100 
2000.01.18D22:10:01.884929254 335.0751 500 
..


We can retrieve these individual files by using [get](https://code.kx.com/q/ref/get/):

In [19]:
10 sublist get `:tab/timestamp  //returns a vector containing in the column
10 sublist get `:tab/price     
get `:tab/.d                   //our column orderings!

2000.01.01D01:56:54.531680879 2000.01.01D13:34:24.561852264 2000.01.02D14:39:..
311.5894 198.4798 234.7829 170.0476 487.053 455.2298 416.9192 220.4627 383.38..
`timestamp`price`size


##### Exercise 

Create a table with `n:200` records, with the following columns: 
   * orderID: 10 characters of random alphanumeric values 
   * price: between 10-20.5 
   * size: between 1000-5000
 
Save this table down as a splayed table `exSplay`. 

Hints: *(Hightlight to see)*:<font color="white">.Q.a and .Q.n store lowercase characters and numbers. 10 cut will split a list at every 10th index</font>

In [ ]:
n:200 
.Q.a,.Q.n                                              //.Q.a and .Q.n store lowercase characters and numbers
`:exSplay/ set ([]orderID: 10 cut (10*200)?.Q.a,.Q.n;  //10 cut splits on every 10th index
                    price: 10+n?10.5; size: 1000 + n?4001)

In [ ]:
key `:exSplay  //notice that our string column is a nested column - orderID, orderID# 

In [23]:
//your answer here 
n:200
exSplay:([]orderID: 10 cut (10*200)?.Q.a,.Q.n; price: 10+n?10.5; size: 1000 + n?4001)
`:exSplay/ set exSplay
\ls exSplay

:exSplay/
"orderID"
"orderID#"
"price"
"size"


## Creating a splayed table with symbol columns 
kdb+ requires enumeration to be applied to symbol columns if we want to persist splay tables on disk. 

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Remember for a table to be saved as splayed, it should be un-keyed and enumerated if a symbol column exists in the table!</i></p>

Let's use that now to enumerate symbol data in our `trade` table  then save to disk splayed. 

In [56]:
trade:([]timestamp:asc n?.z.P;sym:n?`BAC`BTU`DIS`GE`T;price:n?500f; size:n?100 200 500 1000;ex:n?`NYSE`NASDAQ`LSE`JPX)
colsToEnum:`sym`ex
10#trade
meta trade 

timestamp                     sym price    size ex    
------------------------------------------------------
2000.01.09D07:49:49.465930215 BTU 211.8357 100  NASDAQ
2000.03.20D12:16:30.294932286 BTU 368.0263 200  LSE   
2000.06.21D13:11:18.317291198 T   417.7496 1000 JPX   
2000.06.22D12:26:36.408815218 BTU 414.8397 200  JPX   
2000.12.02D08:35:15.578195632 BAC 199.3758 100  LSE   
2001.08.26D03:12:59.109603248 DIS 293.0002 1000 LSE   
2001.08.27D19:22:03.784540368 DIS 2.390399 100  LSE   
2001.11.25D14:29:09.966349560 BTU 447.2774 500  NASDAQ
2002.01.30D05:00:51.975209712 DIS 246.2874 200  LSE   
2002.02.21D21:28:18.946308008 T   143.4259 500  NYSE  
c        | t f a
---------| -----
timestamp| p   s
sym      | s    
price    | f    
size     | j    
ex       | s    


In [55]:
//the below is common way to dynamically apply updates to a specified column list
show updClause:colsToEnum!{(?;enlist `:manualSym;x)} each colsToEnum; //the update clause 

sym| ? ,`:manualSym `sym
ex | ? ,`:manualSym `ex 


In [ ]:
show t: ![trade;();0b;updClause]   //functional update

In [ ]:
type each first t  //we can see the enumeration was successful!
t[`sym]            //or can inspect directly

In [ ]:
enumTab:{[t]
        colsToEnum:exec c from meta[t] where t in"sS";
        //lets build up our enumeration for each, remember `:manualSym?<symList> will  
        //enumearate symList aginst the manualSym file 
        updClause:colsToEnum!{(?;enlist `:manualSym;x)} each colsToEnum;
        //now we'll do a functional update 
        ![t;();0b;updClause]
 }


In [ ]:
`:trade/ set enumTab[trade] 


Unfortunately, we can't use the above method as we need to enumerate the symbol columns:

In [ ]:
`:trade/ set trade   //will return type error as sym columns are not enumerated

Let's check if our `trade` table is enumerated using `first`. We should see ``` `sym$`DIS``` in the `sym` column and ``` `sym$`NASDAQ ``` in the `ex` column

In [57]:
first trade

timestamp| 2000.01.09D07:49:49.465930215
sym      | `BTU
price    | 211.8357
size     | 100
ex       | `NASDAQ


##### `.Q.en` 
Luckily, there is already exists a function within the .Q namespace which can help us to enumerate a table! 

The  function [`.Q.en.`](https://code.kx.com/q/ref/dotq/#qen-enumerate-varchar-cols) enumerates all columns of Symbols  against a file called `sym` in a specified directory passed as an input. The sym file will contain the enumeration of all symbol columns.

Syntax: ```.Q.en[`:directoryToSaveEnumerationSymFile;tableToEnumerate]```

Once this function has been executed, it automatically creates a sym file, if not already there. The table returned from the function is enumerated, but we haven't updated the original table to be enumerated.

For example, the following code shows how a table containing a symbol column can be saved to disk as a splayed table:

In [58]:
`:tradesplay/trade/ set .Q.en[`:tradesplay/] trade

:tradesplay/trade/


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>A limitation of <code>.Q.en</code> is that we cannot change the enumeration filename from `sym`.If we want to change the enumeration filename, we can use <a href="https://code.kx.com/q/ref/dotq/#qens-enumerate-against-domain">.Q.ens</a></i></p>

In [59]:
`:tradesplay/newtrade/ set .Q.ens[`:tradesplay/;trade;`mysym] 
system"ls tradesplay/"
//system"ls tradesplay/"

:tradesplay/newtrade/
"mysym"
"newtrade"
"sym"
"trade"


##### Exercise 
Use `.Q.en` to enumerate the table <code>([]s:10?\`1;s2:10?\`2;s3:10?\`3)</code> in the enumDB directory - don't save the table splayed. 

In [60]:
.Q.en[`:enumDB;([]s:10?`1;s2:10?`2;s3:10?`3)]
get `:enumDB/sym

s s2 s3 
--------
e lo aib
k mh ekk
d ff dhe
e dg pjh
b ge hbk
d ia cdf
p ce lol
l dp edn
l lf oko
k eh kmg
`o`m`h`k`e`c`d`eg`ap`bd`mh`ge`ol`en`cn`pd`ab`emn`bon`boi`kac`gbp`mpc`bhc`ikh`..


In [61]:
// your answer here
//show table1:([]s:10?`1;s2:10?`2;s3:10?`3)
.Q.en[`:enumDB;table1]
//\ls enumDB
get `:enumDB/sym
\ls tradesplay/newtrade

s s2 s3 
--------
d of mip
l gm jch
h kl pnp
l dn bhk
n hn ogm
f kf jcl
a pg omh
n nh pje
d di ajc
j jg ggh
`o`m`h`k`e`c`d`eg`ap`bd`mh`ge`ol`en`cn`pd`ab`emn`bon`boi`kac`gbp`mpc`bhc`ikh`..
"ex"
"price"
"size"
"sym"
"timestamp"


# Operating on a splayed table

We can manipulate a splayed table using the same qSQL methods as we did with tables in memory, with the exception of `update` and `delete` command which cannot modify the table. When referring to the splayed table, we can use it's file handle. 

Let's look at some examples:

In [62]:
select max price from `:tradesplay/trade/ //using select to calculate the max price
`size xdesc `:tradesplay/trade/           //ordering the table in descending order wrt size
get `:tradesplay/trade                    //now ordered on disk!

price   
--------
499.9135
:tradesplay/trade/
timestamp                     sym price    size ex
--------------------------------------------------
2000.06.21D13:11:18.317291198 m   417.7496 1000 eg
2001.08.26D03:12:59.109603248 k   293.0002 1000 d 
2002.05.26D22:25:52.785482080 m   455.1672 1000 c 
2003.06.27D15:29:10.113891728 e   202.301  1000 c 
2004.03.04D04:48:57.639236784 k   217.4016 1000 eg
2004.06.28D18:37:01.571758832 o   335.4528 1000 eg
2005.07.11D21:42:47.655702880 h   360.6626 1000 ap
2006.04.20D07:07:57.066013056 k   406.3522 1000 c 
2007.01.29D10:14:09.000577152 o   129.8229 1000 ap
2007.04.18D04:04:53.197767648 h   445.286  1000 c 
2008.03.04D11:30:44.301784864 k   109.458  1000 eg
2008.03.11D18:03:08.143979520 o   430.8923 1000 ap
2008.11.02D05:35:23.838847872 h   283.8631 1000 d 
2009.09.07D07:59:25.421223744 o   294.4023 1000 d 
2010.11.04D18:46:16.330805248 k   129.6993 1000 c 
2011.09.25D12:46:20.337865152 e   43.54202 1000 d 
2013.08.06D19:26:41.240546624 m   41

However, we cannot delete or update these tables using the file handle:

In [64]:
//delete size from `:tradesplay/trade

In [ ]:
update newCol:size from `:tradesplay/trade

##### Exercise 
Using the filehandle, calculate the average price broken down by sym from the on disk splayed trade table. 

In [ ]:
select avg price by sym from `:tradesplay/trade

In [65]:
//your answer here
select avg price by sym from `:tradesplay/trade

sym| price   
---| --------
e  | 268.3158
h  | 262.4072
k  | 206.5913
m  | 234.0842
o  | 252.7041


## Updating Splayed tables

It is not possible to use qSQL to update data in persisted splayed tables on disk. 

### Adding more rows 
If we wanted to append a record to a splayed table we can use `upsert`. Let's look at a smaller table so we can see if we have added new records to the splayed table.

In [66]:
`:tradesplay/tnew/ set .Q.en[`:tradesplay/tnew;] ([]s:`JPM`IBM`MSFT;price:10 20 30;size:100 200 300) //creating a splayed table
get `:tradesplay/tnew

:tradesplay/tnew/
s    price size
---------------
JPM  10    100 
IBM  20    200 
MSFT 30    300 


In [67]:
`:tradesplay/tnew/ upsert .Q.en[`:tradesplay/tnew;]([]s:`KX`GS;price:40 50;size:400 500)
get `:tradesplay/tnew

:tradesplay/tnew/
s    price size
---------------
JPM  10    100 
IBM  20    200 
MSFT 30    300 
KX   40    400 
GS   50    500 



We can also use [`upsert`](https://code.kx.com/q/ref/upsert/) to create a splayed table. If the file doesn't exist, it will act as `set`. The right hand argument is the file handle that we will write to. In the example below we will create a new splayed table using `upsert`.

In [68]:
`:tmp/ upsert ([] a:10 20 30; b:1.1 2.2 3.3)

:tmp/


In [69]:
get `:tmp

a  b  
------
10 1.1
20 2.2
30 3.3


The keyword `insert` can be used to insert data into a table but unlike `upsert` it will not create a non existant table:

In [71]:
`:tmp/ insert ([]a: 10 20 30;b: 1.1 2.2 3.3)      //inserting into an existing table
//`:newTmp/ insert ([]a: 10 20 30;b: 1.1 2.2 3.3)   //can't insert and create a table

6 7 8


### Adding more columns
To add a new column to a splayed table, we will first have to get the row count from any column file. We will then have to modify the `.d` file so it contains the new columns.

In [72]:
show n:count get `:tradesplay/tnew/s          //getting the number of rows
`:tradesplay/tnew/ex set n?("N";"O")          //creating a new column called exchange
get `:tradesplay/tnew                         //we have made the column but it's not shown in our table yet

:tradesplay/tnew/ex
s    price size
---------------
JPM  10    100 
IBM  20    200 
MSFT 30    300 
KX   40    400 
GS   50    500 
5


In [74]:
`:tradesplay/tnew/.d set get[`:tradesplay/tnew/.d],`ex   //new column added to .d file
get `:tradesplay/tnew                         //new column availble in  table

:tradesplay/tnew/.d
s    price size ex ex
---------------------
JPM  10    100  N  N 
IBM  20    200  N  N 
MSFT 30    300  N  N 
KX   40    400  O  O 
GS   50    500  O  O 


What happens when you re-run the above code cell a few times?

### Deleting from splayed tables
We can also "delete" a column by removing the file and modify the `.d` file to reflect that it doesn't exist anymore.

In [ ]:
system"rm tradesplay/tnew/ex" //removing file 
`:tradesplay/tnew/.d set get[`:tradesplay/tnew/.d] except `ex  //changing .d file to remove column 
get `:tradesplay/tnew //returning new table

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:12px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Removing columns from the .d files is a sort-of "safe" or soft-deletion to do before clearing off the actual data! We can test the impact on users before actually removing the data from the disk, just in case!</i></p>

##### Exercise
Add a new column `timestamp` that contains randomly generated timestamps to the `tnew` splayed table. 

In [ ]:
n:count get `:tradesplay/tnew/                                //getting the number of rows
`:tradesplay/tnew/timestamp set n?.z.p                         //creating a new column
`:tradesplay/tnew/.d set distinct get[`:tradesplay/tnew/.d],`timestamp    //using distinct to avoide column duplication
get `:tradesplay/tnew                                         //new column is added to the table

In [82]:
// your answer here
//\ls tradesplay/tnew
show total:count get `:tradesplay/tnew/size
show timestamp:total?.z.p

`:tradesplay/tnew/timestamp set timestamp
`:tradesplay/tnew/.d set get[`:tradesplay/tnew/.d],`timestamp


:tradesplay/tnew/timestamp
:tradesplay/tnew/.d
5
2017.07.14D19:57:55.435412032 2022.10.24D20:27:39.771329792 2008.08.29D10:12:..


In [83]:
\ls tradesplay/tnew

"ex"
"price"
,"s"
"size"
"sym"
"timestamp"


Modify the type of the `price` column in the splayed table to be a `float` rather than a long. 


In [ ]:
meta get `:tradesplay/tnew

In [ ]:
@[`:tradesplay/tnew;`price;`float$]      //modifying the table directly on disk

In [ ]:
meta get `:tradesplay/tnew

In [89]:
//your answer here 
@[`:tradesplay/tnew;`price;`float$] 

:tradesplay/tnew


# Loading splayed tables
They can be loaded into a q process that is started with a directory as argument or during start-up it
loads a directory using `\l` command. In this case the directory argument is considered as the home
directory of the database.

In [ ]:
\l tradesplay
trade

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Loading a historical database (HDB) always changes the current directory to the HDB directory! Ensure you move back to the original directory or restart the kernel before doing this task</i></p>

In [ ]:
\pwd
system"cd ../" //can also run this to return to original directory
\pwd

We can only have one directory loaded into a kdb+/q process at a given time. In some cases to work around this limitation, people will use [symbolic links](https://en.wikipedia.org/wiki/Symbolic_link) (aka symlinks) to link databases. 

# Guidance on the  sym file

The sym file is a kdb+ binary file containing the list of symbols from all splayed and partitioned tables. During the enumeration process, all columns of symbol type are converted to enumerations against the sym file, after new symbols are added to the sym file. Therefore, the sym file contains a list of unique values. Since the sym file is a kdb+ binary file, it can be read with the [get](https://code.kx.com/q/ref/get/#get) function.

The sym file is located at the same level in our HDB directory as the folder for the splayed table itself. We will look at the `tradesplay` table that we saved down earlier with `.Q.en`: 

In [90]:
key `:tradesplay

`s#`mysym`newtrade`sym`tnew`trade


Looking at the data stored in this:

In [91]:
get `:tradesplay/sym

`BTU`T`BAC`DIS`GE`NASDAQ`LSE`JPX`NYSE


In [92]:
get `:tradesplay/trade

timestamp                     sym  price    size ex
---------------------------------------------------
2000.06.21D13:11:18.317291198 IBM  417.7496 1000   
2001.08.26D03:12:59.109603248 KX   293.0002 1000   
2002.05.26D22:25:52.785482080 IBM  455.1672 1000   
2003.06.27D15:29:10.113891728 GS   202.301  1000   
2004.03.04D04:48:57.639236784 KX   217.4016 1000   
2004.06.28D18:37:01.571758832 JPM  335.4528 1000   
2005.07.11D21:42:47.655702880 MSFT 360.6626 1000   
2006.04.20D07:07:57.066013056 KX   406.3522 1000   
2007.01.29D10:14:09.000577152 JPM  129.8229 1000   
2007.04.18D04:04:53.197767648 MSFT 445.286  1000   
2008.03.04D11:30:44.301784864 KX   109.458  1000   
2008.03.11D18:03:08.143979520 JPM  430.8923 1000   
2008.11.02D05:35:23.838847872 MSFT 283.8631 1000   
2009.09.07D07:59:25.421223744 JPM  294.4023 1000   
2010.11.04D18:46:16.330805248 KX   129.6993 1000   
2011.09.25D12:46:20.337865152 GS   43.54202 1000   
2013.08.06D19:26:41.240546624 IBM  41.1192  1000   
2014.01.04D1

If our sym file becomes damamged or corrupted this is reflected in our table: 

In [ ]:
//NEVER DO THIS IN REAL LIFE
`:tradesplay/sym set neg[count s]?s:get `:tradesplay/sym  //randomly resampling our symbol enumeration 
sym: get `:tradesplay/sym
get `:tradesplay/trade                        //our trade table symbol columns are how misordered

Thus the validity of the data is highly dependent on the sym file - this is why it should be backed up regularly. 

**A great resource for further reading on the sym file is the [Working with sym files](https://code.kx.com/q/wp/symfiles/) whitepaper.**

# Considerations on splayed tables 

##### Limitations

There are some **limitations** with splayed tables:
1. Keyed tables cannot be splayed. 
2. All symbol columns must be enumerated

##### Nested Columns

In cases where we are saving simple lists of lists down to disk, two files are created on disk - `nestedCol` and `nestedCol#`. The first file has the entire column stored as a single continuous list, and the second file (`#`) has a list of indexes at which to cut to reproduce the nested list.

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>A common example of where this occurs are any string columns, since they're really just list of characters!  </i></p>

##### Anymap columns 

Since version 3.6 the [anymap](https://code.kx.com/q/releases/ChangesIn3.6/#anymap) type was introduced as a means to allow the storage and efficient retrieval of nested on disk structures that are not uniform (i.e. all of the same type). Similar to the nested columns, saving an anymap column produces many files, the first two `name` and `name#` are similar to what we saw with *Nested Columns*. 

In [93]:
`:simpleAnyMap set (1 2; 2 3;2 3f;2 4e)  //anything with a type that resolves to 0h saves as anyMap

:simpleAnyMap


In [94]:
get `:simpleAnyMap                       //retrieving the saved data 
type get `:simpleAnyMap                  //77h is the type for anymap
type each get `:simpleAnyMap             //each item still has it's own type 

1 2
2 3
2 3
2 4
77h
7 7 9 8h


In [95]:
f where (f:key `:.) like "simpleAnyMap*"  //the files that were saved for simpleAnyMap

`simpleAnyMap`simpleAnyMap#


There is a third column created when the data we are saving has symbols within it - `name##`. This is a custom enumeration for the symbols present in our nested list. This is denumerated (therefore always copied) on access. 

In [96]:
`:symbolAnyMap set (`a`b!1 2; `d`e!2 3;`e`c!2 3f;2 4e)

:symbolAnyMap


In [97]:
get `:symbolAnyMap               
type get `:symbolAnyMap
type each get `:symbolAnyMap

`a`b!1 2
`d`e!2 3
`e`c!2 3f
2 4e
77h
99 99 99 8h


In [98]:
f where (f:key `:.) like "symbolAnyMap*"  //extra ## file 

`symbolAnyMap`symbolAnyMap#`symbolAnyMap##


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:12px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Memory considerations when using anymap - the <code>name#</code> stays mapped (therefore present in process memory) for as long as there is any ref to object mapped within the structure. </i></p>

##### Quiz Time!
Try the splayed table exercises to test your knowledge and to create and update splayed tables yourself!